# Real-time Sentiment Analysis

This project focuses on the sentiments around the presidential candidates of the ongoing 2020 American presidential election.
<ul>
    <li>First we create a real-time tweet stream</li>
    <li>We extract tweets involving the presidential candidates</li>
    <li>Store these tweets.</li>
    <li>determine the sentiments of these stored tweets.</li>
    <li>visualize these sentiments to campare the popularity of each candidate.</li>
</ul>

It should be noted that prior to starting this project, we have initially obtained a twetter key from twitter in order to access the tweets.

To vizualize the sentiments in real time __[see]()__

In [1]:
import tweepy

## Create a twitter API

Use the CONSUMER_KEY, CONSUMER_SECRET, ACCESS_TOKEN and ACCESS_TOKEN_SECRET to generate an API to acces twitter. To understand how to get this, please see this twitter __[tutorial](https://developer.twitter.com/en/docs/authentication/overview#:~:text=Your%20app's%20Consumer%20API%20Keys,the%203%2Dlegged%20OAuth%20process)__

In [2]:
def create_api():
    ''' Authenticate to Twitter
        auth = tweepy.OAuthHandler("CONSUMER_KEY", "CONSUMER_SECRET")
        auth.set_access_token("ACCESS_TOKEN", "ACCESS_TOKEN_SECRET")
    '''
    auth = tweepy.OAuthHandler("9ZhwL8JM7pBIyJMab9UC9GT6B", "eQKfKZ6Lqbay4toUh48YJp29lj22P8ndWjBYKfbMeMeDzezuSm")
    auth.set_access_token("996363099983753216-aIo7OjMl5NihOggYv6azmaPII5CBCtI", "1VgOEfy6zhHouvWLmKKfVblqRakVZk18wiqvcFkVRLodD")

    # Create an test API object
    api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)
    try:
        api.verify_credentials()
        print("Authentication OK")
    except:
        print("Error during authentication")
    
    return auth, api


## Get real-time tweets

In [3]:
!pip install textblob

In [4]:
from tweepy import Stream
from tweepy import StreamListener
import json
from textblob import TextBlob     # Sentiment classifyer library

In [5]:
import re
import csv

In the next step, we open a CSV file to store our dataset in realtime, after which we would then create real-time visualizations to analyze the election candidate trends.

In [6]:
biden = 0
trump = 0

header_name = ['Trump', 'Biden']
with open('sentiment.csv', 'w') as file:
    writer = csv.DictWriter(file, fieldnames = header_name)
    writer.writeheader()

We create a listener, which we will then use in extracting key words from streaming tweets. In this case, since we are trying to monitor the sentiments of political candidates, we will be monitoring the names of the two candidates, i.e.trump and biden.

In [7]:
class listener(StreamListener):
    
    def on_data(self, data):
        raw_tweets = json.loads(data)
        try:
            cleaned_tweets = raw_tweets['text']
            cleaned_tweets = ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", cleaned_tweets).split()) # remove numbers, special characters and punctuation from tweets
            cleaned_tweets = ' '.join(re.sub('RT',' ',cleaned_tweets).split()) # remove RT
            cleaned_tweets = cleaned_tweets.lower()   #make tweet lower case
            
            blob = TextBlob(cleaned_tweets.strip())
            
            global trump
            global biden
            
            trump_sentiment = 0
            biden_sentiment = 0
            
            
            # do a sentement analysis sentence by sentence
            # If the polarity is less than 0, this is a negative sentiment. If its more than 0 its positive, else, it is neutral
            for sent in blob.sentences:
                if "trump" in sent and "biden" not in sent:
                    trump_sentiment = trump_sentiment + sent.sentiment.polarity  # get the sumation of all trump's sentiments
                else:
                    biden_sentiment = biden_sentiment + sent.sentiment.polarity  # get the summation of all biden's sentiments
            
            trump = trump + trump_sentiment   # get the cummulative sum of all Trump's sentiment for this invocation of the listener
            biden = biden + biden_sentiment   # get the cummulative sum of all biden's sentiment for this invocation of the listener
            
            # store the information in the CSV file
            with open('sentiment.csv', 'a') as file:
                writer = csv.DictWriter(file, fieldnames = header_name)
                
                dataset = {
                    
                    'Trump': trump,
                    'Biden': biden
                }
                writer.writerow(dataset)
            
            print(cleaned_tweets)
        
        except:
            print('Error got')
        
        
    def on_error(self, status):
        print(status)

Get the twitter api

In [8]:
auth, api = create_api()

Authentication OK


Initialize the listener

In [9]:
twitter_stream = Stream(auth, listener())

Filter out tweets with the word biden or trump in it

In [10]:
twitter_stream.filter(track = ['trump', 'biden'])

trump s pardon of mike flynn is a final act of venal malevolent retributive warfare waged on the country that humiliat
today trump could have urged americans to behave responsibly in light of the pandemic he could have asked congress to mov
sleepy joe biden had a whopping 1 000 viewers for his special live thanksgiving message where were the other 79 999 00
we need to strike a middle course one that encourages china s rise said biden s pick for national security advisor
michigan kraken they had multiple spikes in biden votes that were physically impossible for the machines to count f
it s unclear if biden s administration would pursue that effort but generally pardon case law suggests the pardoner n
breaking a judge in nevada has agreed to let the trump campaign present its evidence that fraud and illegalities plagued th
i thought something like that had happened twitter suspends anyone that is important has a big following and sp
cdc ready for a biden transition this is what we ve bee

KeyboardInterrupt: 